In [1]:
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu128
!pip install unsloth
!pip install transformers
!pip install --no-deps trl

Looking in indexes: https://download.pytorch.org/whl/cu128
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.7/405.7 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 148.6 MB/s eta 0:0

In [2]:
import random
import numpy as np
import torch

random.seed(123)
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)

torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")

max_seq_length = 4096
dtype = None ## Auto detect
load_in_4bit = True

/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)


In [3]:
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
base_model = "unsloth/tinyllama-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=base_model,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    device_map="auto",
)

==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


But with kaiokendev's RoPE scaling of 2.0, it can be magically be extended to 4096!


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [5]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear4bit(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNo

In [6]:
tokenizer

LlamaTokenizerFast(name_or_path='unsloth/tinyllama-bnb-4bit', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r=32,            ## LoRA rank (8/16/32/64 are common)
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,     ## Usually r or 2*r
    bias="none",
    use_gradient_checkpointing=False,
    random_state=123
)

Unsloth 2026.1.4 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


In [8]:
model.print_trainable_parameters()

trainable params: 25,231,360 || all params: 1,125,279,744 || trainable%: 2.2422


In [9]:
for name, param in model.named_parameters():
  print(f"Layer name: {name} | Shape: {param.shape}  |  Requires grad: {param.requires_grad}")

Layer name: base_model.model.model.embed_tokens.weight | Shape: torch.Size([32000, 2048])  |  Requires grad: False
Layer name: base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight | Shape: torch.Size([2097152, 1])  |  Requires grad: False
Layer name: base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight | Shape: torch.Size([32, 2048])  |  Requires grad: True
Layer name: base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight | Shape: torch.Size([2048, 32])  |  Requires grad: True
Layer name: base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight | Shape: torch.Size([262144, 1])  |  Requires grad: False
Layer name: base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight | Shape: torch.Size([32, 2048])  |  Requires grad: True
Layer name: base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight | Shape: torch.Size([256, 32])  |  Requires grad: True
Layer name: base_model.model.model.layers.0.self_attn.v

In [10]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"Total trainable parameters: {trainable_params}")
print(f"Total parameters: {total_params}")
print(f"Percent: {(trainable_params / total_params) * 100 :.2f}%")

Total trainable parameters: 25231360
Total parameters: 640837632
Percent: 3.94%


In [11]:
next(model.parameters()).device

device(type='cuda', index=0)

In [12]:
next(model.parameters()).dtype

torch.float16

In [13]:
from peft import PeftModel

print(isinstance(model, PeftModel))

True


In [14]:
model.peft_config

{'default': LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping={'base_model_class': 'LlamaForCausalLM', 'parent_library': 'transformers.models.llama.modeling_llama', 'unsloth_fixed': True}, peft_version='0.18.1', base_model_name_or_path='unsloth/tinyllama-bnb-4bit', revision=None, inference_mode=False, r=32, target_modules={'v_proj', 'o_proj', 'down_proj', 'up_proj', 'q_proj', 'gate_proj', 'k_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, alora_invocation_tokens=None, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False)

In [15]:
eos_token = tokenizer.eos_token

alpaca_format = """Below is the instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately comlpetes the request.

### Instruction:
{}

### Input:
{}

## Response:
{}"""

def format_data(examples):
  texts = []
  for instructions, input_text, output in zip(
      examples["instruction"], examples["input"], examples["output"]
  ):
    text = alpaca_format.format(instructions, input_text, output) + eos_token
    texts.append(text)
  return {"text": texts}

dataset = load_dataset("yahma/alpaca-cleaned", split="train")
dataset = dataset.shuffle(seed=123).select(range(750))

dataset = dataset.map(format_data, batched=True, remove_columns=dataset.column_names)

README.md: 0.00B [00:00, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

In [16]:
dataset

Dataset({
    features: ['text'],
    num_rows: 750
})

In [17]:
!pip install -q psutil

In [19]:
sft_config = SFTConfig(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    warmup_ratio=0.1,
    optim="adamw_8bit",
    lr_scheduler_type="cosine",
    logging_steps=20,
    seed=123,
    output_dir="output",
    report_to="none"
)

In [20]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=sft_config,
    train_dataset=dataset,
    dataset_text_field="text",
    packing=True
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/750 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [21]:
import time, psutil

torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [22]:
process = psutil.Process()
train_start_time = time.time()
cpu_ram_before = process.memory_info().rss / (1024 * 1024 * 1024) ## in GB

trainer.train()

train_end_time = time.time()
cpu_ram_after = process.memory_info().rss / (1024 * 1024 * 1024)
peak_gpu_vram_gb = round(torch.cuda.max_memory_reserved() / (1024**3), 3)
cpu_ram_used_gb = round(cpu_ram_after - cpu_ram_before, 3)
total_training_time_min = round((train_end_time - train_start_time) / 60, 3)

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 750 | Num Epochs = 1 | Total steps = 47
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 25,231,360 of 1,125,279,744 (2.24% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
20,2.037400
40,2.044400


In [23]:
print(f"Training Time (in min): {total_training_time_min}")
print(f"Peak GPU VRAM (in GB): {peak_gpu_vram_gb}")
print(f"CPU RAM Used (in GB): {cpu_ram_used_gb}")

Training Time (in min): 3.307
Peak GPU VRAM (in GB): 1.945
CPU RAM Used (in GB): 0.849


In [26]:
model_save = "tinyllama_lora_model"

model.save_pretrained(model_save)
tokenizer.save_pretrained(model_save)

('tinyllama_lora_model/tokenizer_config.json',
 'tinyllama_lora_model/special_tokens_map.json',
 'tinyllama_lora_model/tokenizer.model',
 'tinyllama_lora_model/added_tokens.json',
 'tinyllama_lora_model/tokenizer.json')

In [27]:
model.push_to_hub("tinyllama_lora_model")
tokenizer.push_to_hub("tinyllama_lora_model")

README.md:   0%|          | 0.00/544 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   1%|          | 1.00MB /  101MB            

Saved model to https://huggingface.co/tinyllama_lora_model


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ora_model/tokenizer.model: 100%|##########|  500kB /  500kB            